In [2]:
# Add a Shortcut from CoDeRs to you drive to access it (in Google Drive, under "Organize")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Edge Detection using OpenCV

In [12]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import glob, os
from google.colab.patches import cv2_imshow


os.chdir("/content/drive/MyDrive/CoDeRs/Vision/ImagesFromNao")


for f in glob.glob("*.j*"):
  print(f)
  src = cv2.imread("/content/drive/MyDrive/CoDeRs/Vision/ImagesFromNao/" + f, cv2.IMREAD_COLOR)
  print("SHAPE: ", src.shape)
  if src.shape[0] != 480:
    # skip pictures that weren't taken by the NAO
    continue

  #Transform source image to gray if it is not already
  if len(src.shape) != 2:
      gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
  else:
      gray = src

  # add smoothing
  # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  # blurred = cv2.bilateralFilter(gray, 5, 175, 175)
  # ret,blurred = cv2.threshold(blurred,100,255,0)

  edges = cv2.Canny(gray, 50, 150)
  contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  # Separate closed and open contours
  closed_contours = []
  open_contours = []

  for i, contour in enumerate(contours):
      # Check if contour has a parent
      if hierarchy[0][i][3] == -1:
          closed_contours.append(contour)
      else:
          area = cv2.contourArea(contour)
          if area > 10:
            # print("Area: ", area)
            epsilon = 0.03 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if (len(approx) >= 5) and (len(approx) < 7):
              mask = np.zeros(src.shape[:2], np.uint8)
              cv2.drawContours(mask, contour, -1, 0, 2)
              mean = cv2.mean(cv2.cvtColor(src, cv2.COLOR_BGR2GRAY), mask=mask)
              # print("Mean color: ", mean)
              if (mean[0] < 105):
                # print(cv2.isContourConvex(contour))
                open_contours.append(contour)

  # Draw closed contours on original image
  cv2.drawContours(src, closed_contours, -1, (0, 255, 0), 2)

  # Draw open contours on original image
  cv2.drawContours(src, open_contours, -1, (0, 0, 255), 2)


  median = [0,0]
  for contour in open_contours:
    M = cv2.moments(contour)
    #print(M)

    # Calculate centroid
    if M["m00"] != 0:  # Ensure that the contour has area
        centroid_x = int(M["m10"] / M["m00"])
        centroid_y = int(M["m01"] / M["m00"])
        median[0] += centroid_x
        median[1] += centroid_y
        #print(centroid_x)
        #print(centroid_y)
    else:
        centroid_x, centroid_y = 0, 0  # Default to (0, 0) if the contour has no area


    # Draw the centroid on the original image
    cv2.circle(src, (centroid_x, centroid_y), 5, (0, 255, 0), -1)  # Draw a green circle at the centroid


  cv2_imshow(src)


Output hidden; open in https://colab.research.google.com to view.

### To Do:

1.   Recursively look for closed shapes with smaller areas until 2 within a reasonable distance from each other are found (this second distance also decreases when looking for the ball further away)
2.   Add a color check: the closed shapes found should be black, or darker than the surrounding area



# Function Script

In [ ]:
def detect_ball(image, min_area = 350):
    """
    INPUTS:
    image - one of the two robot cameras' captured image
    min_area - the minimal sizee of closed shapes to consider when calculating centroids, to remove noise

    OUTPUT:
    median - list, containing the X and Y coordinate of the center of the ball
    """
    src = cv2.imread(image, cv2.IMREAD_COLOR)

    # Transform source image to grayscale if it is not already
    if len(src.shape) != 2:
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    else:
        gray = src

    # Apply blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    blurred = cv2.bilateralFilter(gray, 5, 175, 175)

    # Find contours
    edges = cv2.Canny(blurred, 50, 150)
    contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Separate closed and open contours
    open_contours = []
    ball_contours = []


    for i, contour in enumerate(contours):
        # Check if contour has a parent
        if hierarchy[0][i][3] == -1:
            open_contours.append(contour)
        else:
            # check if area of closed contour is bigger than the specified minimum
            area = cv2.contourArea(contour)
            if area > min_area:
              # check if the contoured shape has around 5 edges
              epsilon = 0.03 * cv2.arcLength(contour, True)
              approx = cv2.approxPolyDP(contour, epsilon, True)
              if (len(approx) >= 5) and (len(approx) < 7):
                ball_contours.append(contour)
                # an attempt at only keeping black shapes, although differences in light
                # intensity make it tricky. Currently not used.
                # mask = np.zeros(src.shape[:2], np.uint8)
                # cv2.drawContours(mask, contour, -1, 0, 2)
                # mean = cv2.mean(cv2.cvtColor(src, cv2.COLOR_BGR2GRAY), mask=mask)
                # print("Mean color: ", mean)
                # if (mean[0] < 105):
                #   ball_contours.append(contour)

    median = [0,0]
    M = {"m00": 0}

    for contour in ball_contours:
        M = cv2.moments(contour)
        # Calculate centroid
        if M["m00"] != 0:  # Ensure that the contour has area
            centroid_x = int(M["m10"] / M["m00"])
            centroid_y = int(M["m01"] / M["m00"])
            median[0] += centroid_x
            median[1] += centroid_y
        else:
            centroid_x, centroid_y = 0, 0  # Default to (0, 0) if the contour has no area
            print("No Ball Detected")

    if len(ball_contours) != 0:
        median = [x / len(ball_contours) for x in median]

    return median


img_path = "/content/output8.jpg"

ball_position = detect_ball(img_path)
print(ball_position)

img = cv2.imread(img_path, cv2.IMREAD_COLOR)




if ball_position != [0,0]:
        # 277 > 240 + 5
        if (ball_position[0]) < (img.shape[1] / 2) - 5:
            print("Ball to the left")

        elif (ball_position[0]) > (img.shape[1] / 2) + 5:
            print("Ball to the right")

        if (ball_position[1]) > (img.shape[0] / 2) + 5:
            print("Ball down")

        elif (ball_position[1]) < (img.shape[0] / 2) - 5:
            print("Ball up")

[426.3333333333333, 159.66666666666666]
Ball to the right
Ball up
